Урок 2. Профилирование пользователей. Сегментация: unsupervised learning (clustering, LDA/ARTM), supervised (multi/binary classification)

2. Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана. Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог с помощью precision_recall_curve, как это делалось на уроке)

In [6]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

In [14]:
news = pd.read_csv("articles.csv")
print(news.shape)
news.head(3)

(27000, 2)


,doc_id,title
0,6,"Заместитель председателяnправительства РФnСергейnИвановnизбран председателем советаnПБК ЦСКАn. Как сообщаетnпресс-службаnармейцев, такое решение было единогласно принято на первом заседании совета клуба. Основной функцией этого органа будет обсуждение текущего состояния, планирование и утверждение стратегии развития клуба на ближайшую перспективу.nnВ состав совета вошли:nГеоргий Полтавченкоn(полномочный представитель президента в Центральном федеральном округе),nАлексей Седовn(генерал-полковник, начальник службы по защите конституционного строя и борьбе с терроризмомnФСБ РФn),nАлександр Новакn(заместитель министра финансов РФ),nВячеслав Полтавцевn(заместитель генерального директора, руководитель блока по взаимодействию с органами власти, социальной и корпоративной политикиnГМК «Норильский никель»n),nЛеонид Исаковичn(генеральный директор ООО «УК «Спортивные проекты»),nОльга Плешаковаn(генеральный директорn«Трансаэро»n),nАлександр Провоторовn, (генеральный директорnОАО «Ростелеком»n), Андрей Лопатов (экс-игрок ЦСКА, чемпион мира, трехкратный чемпион Европы по баскетболу, председатель правленияnбанка «Клиентский»n),nАндрей Ватутинn(президент ПБК ЦСКА)."
1,4896,"Матч 1/16 финала Кубка России по футболу был приостановлен судьей из-за взрыва пиротехнических снарядов, передает корреспондент «Газеты.Ru». Болельщики выбросили на поле петарды. Судья увел команды с поля в подтрибунное помещение. Динамовцы ушли, а торпедовцы остались у кромки поля. Сообщается, что матч остановлен на пять минут.n«Газета.Ru» ведетnонлайн-трансляциюnэтого матча."
2,4897,"Форвард «Авангарда» Томаш Заборский прокомментировал игру своей команды в матче чемпионата КХЛ против «Атланта»n(4:3)n.nn«Мы провели плохой матч в Нижнем Новгороде против «Торпедо» и настраивались, что с первых же минут включимся в работу, — сказал Заборский. — У нас получилось забросить быстрый гол и задать хороший темп поединку. Мы можем играть еще лучше, но, с другой стороны, пять очков на выезде из девяти — это лучше, чем ничего»."


In [19]:
# Вспоминая вебинар хочется отметить наличие странных букв 'n' в местах где должны быть пробелы..
# Попробуем сразу это исправить..
# Надеюсь ничего не поломаем важного

def replace_n(source):
    return source.replace('n', ' ')

news.title = news.apply(lambda x: replace_n(x.title), axis=1)
news.head(3)

,doc_id,title
0,6,"Заместитель председателя правительства РФ Сергей Иванов избран председателем совета ПБК ЦСКА . Как сообщает пресс-служба армейцев, такое решение было единогласно принято на первом заседании совета клуба. Основной функцией этого органа будет обсуждение текущего состояния, планирование и утверждение стратегии развития клуба на ближайшую перспективу. В состав совета вошли: Георгий Полтавченко (полномочный представитель президента в Центральном федеральном округе), Алексей Седов (генерал-полковник, начальник службы по защите конституционного строя и борьбе с терроризмом ФСБ РФ ), Александр Новак (заместитель министра финансов РФ), Вячеслав Полтавцев (заместитель генерального директора, руководитель блока по взаимодействию с органами власти, социальной и корпоративной политики ГМК «Норильский никель» ), Леонид Исакович (генеральный директор ООО «УК «Спортивные проекты»), Ольга Плешакова (генеральный директор «Трансаэро» ), Александр Провоторов , (генеральный директор ОАО «Ростелеком» ), Андрей Лопатов (экс-игрок ЦСКА, чемпион мира, трехкратный чемпион Европы по баскетболу, председатель правления банка «Клиентский» ), Андрей Ватутин (президент ПБК ЦСКА)."
1,4896,"Матч 1/16 финала Кубка России по футболу был приостановлен судьей из-за взрыва пиротехнических снарядов, передает корреспондент «Газеты.Ru». Болельщики выбросили на поле петарды. Судья увел команды с поля в подтрибунное помещение. Динамовцы ушли, а торпедовцы остались у кромки поля. Сообщается, что матч остановлен на пять минут. «Газета.Ru» ведет онлайн-трансляцию этого матча."
2,4897,"Форвард «Авангарда» Томаш Заборский прокомментировал игру своей команды в матче чемпионата КХЛ против «Атланта» (4:3) . «Мы провели плохой матч в Нижнем Новгороде против «Торпедо» и настраивались, что с первых же минут включимся в работу, — сказал Заборский. — У нас получилось забросить быстрый гол и задать хороший темп поединку. Мы можем играть еще лучше, но, с другой стороны, пять очков на выезде из девяти — это лучше, чем ничего»."


In [20]:
users = pd.read_csv("users_articles.csv")
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


### 1. Получаем векторные представления новостей

In [22]:
from gensim.corpora.dictionary import Dictionary
#предобработка текстов
import re
import numpy as np
from nltk.corpus import stopwords
from razdel import tokenize 
import pymorphy2

In [24]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/postas/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [25]:
stopword_ru = stopwords.words('russian')
morph = pymorphy2.MorphAnalyzer()

In [26]:
with open('stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
stopword_ru += additional_stopwords
len(stopword_ru)

776

In [27]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    
    #tokens = list(tokenize(text))
    #words = [_.text for _ in tokens]
    #words = [w for w in words if w not in stopword_ru]
    
    #return " ".join(words)
    return text

cache = {}

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [28]:
%%time
#Запускаем очистку текста. Будет долго...
news.title = news.title.apply(lambda x: clean_text(x), 1)

<ipython-input-27-7ee348d9b386>:15: FutureWarning: Possible nested set at position 39
  text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)


CPU times: user 25 s, sys: 479 ms, total: 25.5 s
Wall time: 26.6 s


In [29]:
%%time
#Запускаем лемматизацию текста. Будет очень долго...
news.title = news.title.apply(lambda x: lemmatization(x), 1)

CPU times: user 3min 4s, sys: 1.45 s, total: 3min 5s
Wall time: 3min 12s


In [34]:
#сформируем список наших текстов, разбив еще и на пробелы
texts = [t for t in news['title'].values]
# Create a corpus from a list of texts
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

In [37]:
%%time
from gensim.models import LdaModel
# Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=25, id2word=common_dictionary)

CPU times: user 48 s, sys: 598 ms, total: 48.6 s
Wall time: 31.8 s


In [38]:
from gensim.test.utils import datapath
# Save model to disk.
temp_file = datapath("model.lda")
lda.save(temp_file)

# Load a potentially pretrained model from disk.
lda = LdaModel.load(temp_file)

In [40]:
# Create a new corpus, made of previously unseen documents.
other_texts = [t for t in news['title'].iloc[:3]]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]
print(other_texts[2])
lda[unseen_doc] 

['форвард', 'авангард', 'томаш', 'заборский', 'прокомментировать', 'игра', 'команда', 'матч', 'чемпионат', 'кхл', 'против', 'атланта', 'плохой', 'матч', 'нижний', 'новгород', 'против', 'торпедо', 'настраиваться', 'первый', 'минута', 'включиться', 'заборский', 'получиться', 'забросить', 'быстрый', 'гол', 'задать', 'хороший', 'темп', 'поединок', 'играть', 'хороший', 'сторона', 'пять', 'очко', 'выезд', 'девять', 'хороший']


[(0, 0.08589854),
 (13, 0.10926985),
 (18, 0.2950962),
 (20, 0.1342095),
 (21, 0.091034845),
 (23, 0.26338568)]

Вот тут я кстати не понял - почему эти документы previously unseen? Те-же документы, что мы и использовали для построения основного корпуса

In [41]:
x=lda.show_topics(num_topics=25, num_words=7,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

#Below Code Prints Only Words 
for topic,words in topics_words:
    print("topic_{}: ".format(topic)+" ".join(words))

topic_0: всё жизнь мозг очень большой женщина хороший
topic_1: восток вирус устойчивость лесной пётр дальний молдавия
topic_2: русский язык знаменитый рука кровь земля всё
topic_3: станция высота метр метро кг польша конструкция
topic_4: сша новый научный исследование район помощь журнал
topic_5: риск оборудование выборы способность вероятно достигать давление
topic_6: сотрудничать стремительно шок индустрия моделирование козак дизайн
topic_7: египет энергетика лёд австралийский праздничный рисунок шкала
topic_8: область наука смерть сотрудник эксперимент военный университет
topic_9: россия российский новый эксперт правительство вопрос решение
topic_10: рубль млрд млн рост банк цена рынок
topic_11: путин россия пациент владимир температура белоруссия прогноз
topic_12: земля американский запуск источник опубликовать сша китай
topic_13: рак писать исследование британский век офицер великобритания
topic_14: фонд взрыв писать улица дом обнаружить помещение
topic_15: доллар налог снизить на

In [42]:
# Векторное представление новости

def get_lda_vector(text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]
    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(25):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [43]:
topic_matrix = pd.DataFrame([get_lda_vector(text) for text in news['title'].values])
topic_matrix.columns = ['topic_{}'.format(i) for i in range(25)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+['topic_{}'.format(i) for i in range(25)]]
topic_matrix.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,6,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.145716,0.349612,0.398238,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0198,0.078591,0.0
1,4896,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.687014,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0000,0.288771,0.0
2,4897,0.086360,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.108877,0.000000,0.0,0.0,0.0,0.294946,0.0,0.134388,0.090947,0.0000,0.263377,0.0
3,4898,0.322321,0.0,0.077352,0.0,0.212724,0.082015,0.0,0.0,0.000000,0.000000,0.108151,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.084674,0.000000,0.0000,0.103202,0.0
4,4899,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.237982,0.000000,0.309546,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.344391,0.0000,0.083334,0.0


### Следующий шаг - векторные представления пользователей

In [44]:
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


In [45]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[['topic_{}'.format(i) for i in range(25)]].values))

In [47]:
def get_user_embedding_mean(user_articles_list):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.mean(user_vector, 0)
    return user_vector

In [49]:
get_user_embedding_mean(users.articles.iloc[33])

array([0.03702068, 0.00223208, 0.02124774, 0.        , 0.11400733,
       0.01808529, 0.        , 0.        , 0.09603326, 0.2193737 ,
       0.02969175, 0.09925126, 0.07262351, 0.02828392, 0.00736249,
       0.00200613, 0.0065883 , 0.07245527, 0.        , 0.        ,
       0.03143075, 0.02933569, 0.08595776, 0.        , 0.01314966])

In [53]:
# Теперь получим эмбединги для всех пользователей и проверим их качество на конкретной downstream-задаче
user_embeddings_mean = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding_mean(x), 1)])
user_embeddings_mean.columns = ['topic_{}'.format(i) for i in range(25)]
user_embeddings_mean['uid'] = users['uid'].values
user_embeddings_mean = user_embeddings_mean[['uid']+['topic_{}'.format(i) for i in range(25)]]
user_embeddings_mean.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,u105138,0.079412,0.000000,0.068065,0.010006,0.043241,0.014478,0.0,0.000000,0.051649,0.098587,0.016064,0.050441,0.119018,0.021061,0.021310,0.000000,0.027870,0.088208,0.001800,0.007813,0.107977,0.063699,0.069075,0.020419,0.000000
1,u108690,0.048520,0.002758,0.036345,0.000000,0.160301,0.009146,0.0,0.009945,0.057758,0.169393,0.057830,0.059027,0.076437,0.010937,0.009545,0.000000,0.002138,0.062071,0.003389,0.000000,0.024485,0.081642,0.093627,0.000000,0.005878
2,u108339,0.014575,0.000000,0.006172,0.008290,0.139602,0.000000,0.0,0.000000,0.089326,0.196877,0.054804,0.048321,0.004387,0.001893,0.026069,0.001782,0.035551,0.093075,0.013388,0.003459,0.108047,0.015761,0.112821,0.006138,0.007735


In [54]:
# Датасет готов - можно попробовать обучить модель. Загрузим нашу разметку
target = pd.read_csv("users_churn.csv")
target.head(3)

,uid,churn
0,u107120,0
1,u102277,0
2,u102444,0


In [55]:
X_mean = pd.merge(user_embeddings_mean, target, 'left')
X_mean.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24,churn
0,u105138,0.079412,0.000000,0.068065,0.010006,0.043241,0.014478,0.0,0.000000,0.051649,0.098587,0.016064,0.050441,0.119018,0.021061,0.021310,0.000000,0.027870,0.088208,0.001800,0.007813,0.107977,0.063699,0.069075,0.020419,0.000000,0
1,u108690,0.048520,0.002758,0.036345,0.000000,0.160301,0.009146,0.0,0.009945,0.057758,0.169393,0.057830,0.059027,0.076437,0.010937,0.009545,0.000000,0.002138,0.062071,0.003389,0.000000,0.024485,0.081642,0.093627,0.000000,0.005878,1
2,u108339,0.014575,0.000000,0.006172,0.008290,0.139602,0.000000,0.0,0.000000,0.089326,0.196877,0.054804,0.048321,0.004387,0.001893,0.026069,0.001782,0.035551,0.093075,0.013388,0.003459,0.108047,0.015761,0.112821,0.006138,0.007735,1


In [56]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import itertools

import matplotlib.pyplot as plt

%matplotlib inline

In [57]:
#разделим данные на train/test
X_mean_train, X_mean_test, y_mean_train, y_mean_test = train_test_split(X_mean[['topic_{}'.format(i) for i in range(25)]], 
                                                    X_mean['churn'], random_state=0)

In [58]:
logreg_mean = LogisticRegression()
#обучим наш пайплайн
logreg_mean.fit(X_mean_train, y_mean_train)

LogisticRegression()

In [59]:
#наши прогнозы для тестовой выборки
preds_mean = logreg_mean.predict_proba(X_mean_test)[:, 1]
preds_mean[:10]

array([0.07797514, 0.04357014, 0.50066897, 0.1065179 , 0.06368553,
       0.12670055, 0.09568324, 0.19065215, 0.12230131, 0.2154521 ])

### Рассчитаем Precision, Recall, F_score

In [60]:
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

In [62]:
precision, recall, thresholds = precision_recall_curve(y_mean_test, preds_mean)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

Best Threshold=0.265747, F-Score=0.645, Precision=0.612, Recall=0.682


In [63]:
roc_auc_score(y_mean_test, preds_mean)

0.9403244374672945